In [63]:
import tvm_users_scatter as tus
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from bokeh.layouts import layout
from bokeh.io import push_notebook
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, HoverTool, PrintfTickFormatter,NumeralTickFormatter, Legend, Toggle, BoxAnnotation, CustomJS
# import psycopg2
from datetime import datetime,timedelta

In [172]:
df = pd.read_csv('data/2018/channel_user_tvm_sessions_by_PT_2018-03.csv')
# df = tus.cleanDF(df)

In [173]:
df.head()

,platform_type,anon_user_id,name,tvm,sessions
0,CTV,e87065f8-2166-405d-947a-0c1a5aba12d3_d5a6f9c76...,Flicks of Fury,872,2
1,CTV,ROKU-YN00WX009869,Stand-up TV,199,3
2,CTV,ROKU-YW00CU314789,Adventure TV,531,11
3,CTV,46067e6b-de7e-43a6-b139-5b3fad10420c_5754ea96a...,Awesomeness TV,2699,14
4,CTV,ROKU-1GN38D081442,IGN,859,27


In [174]:
df['platform_type'].value_counts()

CTV       1117200
Mobile     235882
Name: platform_type, dtype: int64

In [175]:
df.columns = ['platform_type','anon_user_id','Channel','TVM','Frequency']

In [176]:
df['Average Session Duration'] = df['TVM'] / df['Frequency']

In [177]:
dfs = {}
platform_types = ['CTV','Mobile']
for pt in platform_types:
    dfs[pt] = df[df['platform_type']==pt].reset_index(drop=True)

In [178]:
# Remove upper outliers
for pt in platform_types:
    dfs[pt][dfs[pt]['TVM']<np.percentile(dfs[pt]['TVM'],99)].reset_index(drop=True, inplace=True)
    dfs[pt][dfs[pt]['Frequency']<np.percentile(dfs[pt]['Frequency'],99)].reset_index(drop=True, inplace=True)

In [186]:
gs = {pt:{} for pt in platform_types}
for pt in platform_types:
    gs[pt]['mean'] = dfs[pt].groupby('Channel').mean().reset_index()
    gs[pt]['counts'] = dfs[pt].groupby('Channel').count().reset_index().iloc[:,:2]
    gs[pt]['counts'].columns = ['Channel','users']

In [187]:
channels = pd.read_csv('data/2018/pluto_channels.csv')

In [188]:
for pt in platform_types:
    gs[pt]['mean'] = gs[pt]['mean'].merge(channels, left_on='Channel',right_on='name')
    gs[pt]['mean'] = gs[pt]['mean'].merge(gs[pt]['counts'], left_on='Channel',right_on='Channel')

In [191]:
min_users = 300
bad_cats = ['Music + Radio','Pluto']

for pt in platform_types:
    bad_channels = gs[pt]['counts'][gs[pt]['counts']['users']<min_users]['Channel']
    gs[pt]['mean'] = gs[pt]['mean'][np.invert(gs[pt]['mean']['category'].isin(bad_cats))]
    gs[pt]['mean'] = gs[pt]['mean'][np.invert(gs[pt]['mean']['Channel'].isin(bad_channels))]

In [192]:
# Final grooming
gs['Mobile']['mean'] = gs['Mobile']['mean'][gs['Mobile']['mean']['Channel']!='CBSN 3']

#### CTV

In [195]:
p = createViz(gs['CTV']['mean'], 
              xcol='Frequency', ycol='Average Session Duration', zcol='users', scale=8e-4,
              title='Frequency (x) vs. Average Session Duration (y) vs. Users (z) (CTV, Mar 2018)',
              line_type='quadrant', output_format='html', output_fn='htmls/channel_scatterplot_CTV_2018-03.html')

#### Mobile

In [170]:
p = createViz(gs['Mobile']['mean'], 
              xcol='sessions', ycol='asd', zcol='users', scale=3e-3,
              title='Frequency (x) vs. Average Session Duration (y) vs. Users (z) (Mobile, Mar 2018)',
              line_type='quadrant', output_format='html', output_fn='htmls/channel_scatterplot_Mobile_2018-03.html')

#### Define createViz

In [194]:
def createViz(df, xcol, ycol, zcol, output_format='notebook', title="Users vs TVMs", output_fn="users_tvms.html",
              line_type='regression', scale=1e-5):
    if output_format == 'notebook':
        output_notebook()
    elif output_format == 'html':
        output_file(output_fn, title=title)
    else:
        print 'Unknown format'

    source_dict = dict()
    color_lst = ['green','blue','red','black','orange','yellow','pink','grey','turquoise','brown']
    r = lambda: random.randint(0,255)
    h = lambda: '#%02X%02X%02X' % (r(),r(),r())
    color_lst = [color_lst[i] for i,x in enumerate(df['category'].unique())]
    category_lst = sorted(df.category.unique())
    color_mapping = {cat:color for cat,color in zip(category_lst, color_lst)}
    category_colors = [color_mapping[cat] for cat in df.category]
    source = ColumnDataSource(data= dict(
                                x=df[xcol],
                                y=df[ycol],
                                z=df[zcol],
                                zsize=df[zcol]*scale,
                                channel=df['Channel'],
                                color=category_colors,
                                category=df.category
                                )
                            )
    color_dict = {cat:color for cat,color in zip(sorted(df.category.unique()), color_lst)}

    TOOLS = "pan,wheel_zoom,reset,hover,save"

    p = figure(title=title, tools=TOOLS, plot_width=1000, plot_height=600)
    p.scatter(x='x', y='y', size='zsize', source=source, color='color', legend='category')
    p.xaxis[0].axis_label = xcol
    p.yaxis[0].axis_label = ycol


    labels = LabelSet(x='x', y='y', text='channel', level='glyph',
                      x_offset=5, y_offset=5, source=source, render_mode='canvas')
    p.add_layout(labels)

    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [
        ("Channel", "@channel"),
        ("Monthly Frequency", "@x{0.0}"),
        ("Average Session Duration", "@y{0,0}"),
        ("Users", "@z{0,0}")
    ]

    x_min = 0
    y_min = 0
    # This code is to determine xmax and ymax, or xmid and ymid
    if line_type == 'regression':
        n=np.std(df[ycol])/2
        x_max = max(df[xcol])
        slope, intercept = np.polyfit(df[xcol], df[ycol], 1)
        y_max = x_max*slope + intercept
        xx = np.array([x_min, x_max])
        yy = np.array([y_min, y_max])
        p.line(xx, yy, line_width=1, line_color='black')
        p.line(xx, yy-n, line_width=0.5, line_color='grey')
        p.line(xx, yy+n, line_width=0.5, line_color='grey')
    elif line_type == 'quadrant':
        x_mid = df[xcol].mean()
        y_mid = df[ycol].mean()
        x_max = df[xcol].max()
        y_max = df[ycol].max()
        p.line(np.array([x_mid, x_mid]), np.array([0,y_max*1.2]),line_width=1, line_color='grey')
        p.line(np.array([0, x_max*1.2]), np.array([y_mid, y_mid]),line_width=1, line_color='grey')

    p.xaxis.bounds = (x_min*0.75, x_max*1.25)
    p.yaxis.bounds = (y_min, y_max*1.25)

    # legend = Legend(items= [(cat,[scatter_items[cat]]) for cat in df.category.unique()])
    legend = Legend()
    p.add_layout(legend, 'right')
    p.legend.location = "top_left"
    p.legend.label_text_font_size = '8pt'
    p.legend.click_policy="hide"

    # We write coffeescript to link toggle with visible property of labels
    code = '''
    object.visible = toggle.active
    '''
    callback1 = CustomJS.from_coffeescript(code=code, args={})
    toggle1 = Toggle(label="Show/Hide Channel Labels", button_type="success", callback=callback1)
    callback1.args = {'toggle': toggle1, 'object': labels}

    show(layout([p],[toggle1]))
    # handle = show(p, notebook_handle=True)
    # return p